In [3]:
import sys
sys.path.append("/Users/aermolaev/Downloads/data_processing/")

import pandas as pd
import os
from db import connection
from db_utils import DBUtils
import xlrd
from tqdm import tqdm
import datetime
import json
from glob import glob
import zipfile
import requests


path_to_data = "/Users/aermolaev/Downloads/FAOSTAT/*.zip"
path_to_unrar = "/Users/aermolaev/Downloads/data_processing/fao_dataset/data/"

In [4]:
metadata_groups = requests.get("http://fenixservices.fao.org/faostat/api/v1/en/groupsanddomains?section=download").json()['data']
def getMetadata(domain_code):
    url = "http://fenixservices.fao.org/faostat/api/v1/en/metadata/%s" % domain_code
    return requests.get(url).json()['data']

metadata_by_name = { group['domain_name']: getMetadata(group['domain_code']) for group in metadata_groups }


In [5]:
def find(fn, lst):
    for item in lst:
        if fn(item):
            return item
        
def get_add_info(metadata):
    try:
        return find(lambda row: row['metadata_label'] == 'Statistical concepts and definitions', metadata)['metadata_text']

    except:
        return ""
def get_pub_source(metadata):
    try:
        return find(lambda row: row['metadata_label'] == 'Source data', metadata)['metadata_text']
    except:
        return ""

def get_pub_desc(metadata):
    try:
        return find(lambda row: row['metadata_label'] == 'Data description', metadata)['metadata_text']
    except:
        return ""

In [6]:
pd.DataFrame(metadata_groups)['domain_name'].values.tolist()

['Crops',
 'Crops processed',
 'Live Animals',
 'Livestock Primary',
 'Livestock Processed',
 'Production Indices',
 'Value of Agricultural Production',
 'Crops and livestock products',
 'Live animals',
 'Detailed trade matrix',
 'Trade Indices',
 'New Food Balances',
 'Food Balances (old methodology and population)',
 'Commodity Balances - Crops Primary Equivalent',
 'Commodity Balances - Livestock and Fish Primary Equivalent',
 'Food Supply - Crops Primary Equivalent',
 'Food Supply - Livestock and Fish Primary Equivalent',
 'Indicators from Household Surveys (gender, area, socioeconomics)',
 'Suite of Food Security Indicators',
 'Producer Prices - Annual',
 'Producer Prices - Monthly',
 'Producer Price Indices - Annual',
 'Producer Prices - Archive',
 'Consumer Price Indices',
 'Deflators',
 'Exchange rates - Annual',
 'Fertilizers by Nutrient',
 'Fertilizers by Product',
 'Fertilizers archive',
 'Pesticides Use',
 'Pesticides Trade',
 'Land Use',
 'Employment Indicators',
 'Annual 

In [7]:
column_types = [
    # 11 columns
    tuple(["Area Code", "Area", "Item Code", "Item", "ISO Currency Code", "Currency", "Year Code", "Year", "Unit", "Value", "Flag"]),
    tuple(["CountryCode", "Country", "ItemCode", "Item", "ElementGroup", "ElementCode", "Element", "Year", "Unit", "Value", "Flag"]),
    tuple(["Area Code", "Area", "Item Code", "Item", "Element Code", "Element", "Year Code", "Year", "Unit", "Value", "Flag"]),
    tuple(["Country Code", "Country", "Item Code", "Item", "Element Code", "Element", "Year Code", "Year", "Unit", "Value", "Flag"]),
    tuple(["Country Code", "Country", "Source Code", "Source", "Indicator Code", "Indicator", "Year Code", "Year", "Unit", "Value", "Flag"]),
    tuple(["Recipient Country Code", "Recipient Country", "Item Code", "Item", "Donor Country Code", "Donor Country", "Year Code", "Year", "Unit", "Value", "Flag"]),
    # 13 columns
    tuple(["Reporter Country Code", "Reporter Countries", "Partner Country Code", "Partner Countries", "Item Code", "Item", "Element Code", "Element", "Year Code", "Year", "Unit", "Value", "Flag"]),
    # 15 columns
    tuple(["Donor Code", "Donor", "Recipient Country Code", "Recipient Country", "Item Code", "Item", "Element Code", "Element", "Purpose Code", "Purpose", "Year Code", "Year", "Unit", "Value", "Flag"]),
    # for Indicators_from_Household_Surveys_E_All_Data_(Normalized)
    tuple(['Survey Code','Survey','Breakdown Variable Code','Breakdown Variable','Breadown by Sex of the Household Head Code','Breadown by Sex of the Household Head','Indicator Code','Indicator','Measure Code','Measure','Unit','Value','Flag']),
    tuple(['Area Code', 'Area','Source Code','FAO Source','Indicator Code','Indicator','Year Code','Year','Unit','Value','Flag','Note']),
    # ConsumerPriceIndices_E_All_Data_(Normalized).zip
    tuple(['Area Code','Area','Item Code','Item','Months Code','Months','Year Code','Year','Unit','Value','Flag','Note']),
    # Exchange_rate_E_All_Data_(Normalized).zip
    tuple(['Area Code','Area','Item Code','Item','ISO Currency Code','Currency','Year Code','Year','Unit','Value','Flag','Note']),
# Environment_Temperature_change_E_All_Data_(Normalized).zip
tuple(['Area Code','Area','Months Code','Months','Element Code','Element','Year Code','Year','Unit','Value','Flag'])
]

category_files = {
    "Production": [
        "Production_Crops_E_All_Data_(Normalized).zip",
        "Production_CropsProcessed_E_All_Data_(Normalized).zip",
        "Production_Livestock_E_All_Data_(Normalized).zip",
        "Production_LivestockPrimary_E_All_Data_(Normalized).zip",
        "Production_LivestockProcessed_E_All_Data_(Normalized).zip",
        "Production_Indices_E_All_Data_(Normalized).zip",
        "Value_of_Production_E_All_Data_(Normalized).zip"
    ],
    "Trade": [
        "Trade_Crops_Livestock_E_All_Data_(Normalized).zip",
        "Trade_LiveAnimals_E_All_Data_(Normalized).zip",
        "Trade_DetailedTradeMatrix_E_All_Data_(Norm).zip", 
        "Trade_DetailedTradeMatrix_E_All_Data_(Norm).csv",
        "Trade_Indices_E_All_Data_(Norm).zip"
    ],
    "Food Balance": [
        "FoodBalanceSheets_E_All_Data_(Normalized).zip",  
        "FoodBalanceSheets_E_All_Data_(Normalized).csv",
        "CommodityBalances_Crops_E_All_Data_(Normalized).zip",
        "CommodityBalances_LivestockFish_E_All_Data_(Normalized).zip",
        "FoodSupply_Crops_E_All_Data_(Normalized).zip",
        "FoodSupply_LivestockFish_E_All_Data_(Normalized).zip"
    ],
    "Food Security": [
        "Indicators_from_Household_Surveys_E_All_Data_(Normalized).zip",
        "Food_Security_Data_E_All_Data_(Normalized).zip"
    ],
    "Prices": [
        "Prices_E_All_Data_(Normalized).zip",
        "Prices_Monthly_E_All_Data_(Normalized).zip",
        "Price_Indices_E_All_Data_(Normalized).zip",
        "PricesArchive_E_All_Data_(Norm).zip",
        "ConsumerPriceIndices_E_All_Data_(Normalized).zip",
        "Deflators_E_All_Data_(Normalized).zip",
        "Exchange_rate_E_All_Data_(Normalized).zip"
    ],
    "Inputs": [
        "Inputs_FertilizersNutrient_E_All_Data_(Normalized).zip",
        "Inputs_FertilizersProduct_E_All_Data_(Normalized).zip",
        "Inputs_FertilizersArchive_E_All_Data_(Norm).zip",
        "Inputs_FertilizersTradeValues_E_All_Data_(Normalized).zip",
        "Inputs_Pesticides_Use_E_All_Data_(Normalized).zip",
        "Inputs_Pesticides_Trade_E_All_Data_(Norm).zip",
        "Inputs_LandUse_E_All_Data_(Normalized).zip",
        "Employment_Indicators_E_All_Data_(Norm).zip"
    ],
    "Population": [
        "Population_E_All_Data_(Norm).zip"
    ],
    "Investment": [
        "Investment_Machinery_E_All_Data_(Norm).zip",
        "Investment_MachineryArchive_E_All_Data_(Norm).zip",
        "Investment_GovernmentExpenditure_E_All_Data_(Normalized).zip",
        "Investment_CreditAgriculture_E_All_Data_(Normalized).zip",
        "Development_Assistance_to_Agriculture_E_All_Data_(Normalized).zip",
        "Investment_ForeignDirectInvestment_E_All_Data_(Norm).zip",
        "Investment_CountryInvestmentStatisticsProfile_E_All_Data_(Normalized).zip"
    ],
    "Macro-Statistics": [
        "Investment_CapitalStock_E_All_Data_(Normalized).zip",
        "Macro-Statistics_Key_Indicators_E_All_Data_(Normalized).zip"
    ],
    "Agri-Environmental Indicators": [
        "Environment_AirClimateChange_E_All_Data_(Normalized).zip",
        "Environment_Energy_E_All_Data_(Normalized).zip",
        "Environment_Fertilizers_E_All_Data_(Normalized).zip",
        "Environment_LandUse_E_All_Data_(Normalized).zip",
        "Environment_LandCover_E_All_Data_(Normalized).zip",
        "Environment_LivestockPatterns_E_All_Data_(Normalized).zip",
        "Environment_Pesticides_E_All_Data_(Normalized).zip",
        "Environment_Soil_E_All_Data_(Normalized).zip",
        "Environment_Water_E_All_Data_(Normalized).zip",
        "Environment_Emissions_by_Sector_E_All_Data_(Normalized).zip",
        "Environment_Emissions_intensities_E_All_Data_(Normalized).zip",
        "Environment_LivestockManure_E_All_Data_(Normalized).zip",
        "Environment_Temperature_change_E_All_Data_(Normalized).zip"
    ],
    "Emissions - Agriculture": [
        "Emissions_Agriculture_Agriculture_total_E_All_Data_(Normalized).zip",
        "Emissions_Agriculture_Enteric_Fermentation_E_All_Data_(Norm).zip",
        "Emissions_Agriculture_Manure_Management_E_All_Data_(Norm).zip",
        "Emissions_Agriculture_Rice_Cultivation_E_All_Data_(Norm).zip",
        "Emissions_Agriculture_Synthetic_Fertilizers_E_All_Data_(Norm).zip",
        "Emissions_Agriculture_Manure_applied_to_soils_E_All_Data_(Norm).zip",
        "Emissions_Agriculture_Manure_left_on_pasture_E_All_Data_(Norm).zip",
        "Emissions_Agriculture_Crop_Residues_E_All_Data_(Norm).zip",
        "Emissions_Agriculture_Cultivated_Organic_Soils_E_All_Data_(Norm).zip",
        "Emissions_Agriculture_Burning_Savanna_E_All_Data_(Normalized).zip",
        "Emissions_Agriculture_Burning_crop_residues_E_All_Data_(Norm).zip",
        "Emissions_Agriculture_Energy_E_All_Data_(Norm).zip"
    ],
    "Emissions - Land Use": [
        "Emissions_Land_Use_Land_Use_Total_E_All_Data_(Norm).zip",
        "Emissions_Land_Use_Forest_Land_E_All_Data_(Norm).zip",
        "Emissions_Land_Use_Cropland_E_All_Data_(Norm).zip",
        "Emissions_Land_Use_Grassland_E_All_Data_(Norm).zip",
        "Emissions_Land_Use_Burning_Biomass_E_All_Data_(Norm).zip"
    ],
    "Forestry": [
        "Forestry_E_All_Data_(Normalized).zip",
        "Forestry_Trade_Flows_E_All_Data_(Normalized).zip"
    ],
    "ASTI R&D Indicators": [
        "ASTI_Expenditures_E_All_Data_(Normalized).zip",
        "ASTI_Researchers_E_All_Data_(Normalized).zip",
        "ASTI_Research_Spending_E_All_Data_(Normalized).zip"
    ],
    "ASTI Indicators": [
        
    ],
    "Emergency Response": [
        "Food_Aid_Shipments_WFP_E_All_Data_(Normalized).zip"
    ]
    }


file_dataset_names = {
    "ASTI_Expenditures_E_All_Data_(Normalized).zip": "ASTI-Expenditures",
    "ASTI_Researchers_E_All_Data_(Normalized).zip": "ASTI-Researchers",
    "ASTI_Research_Spending_E_All_Data_(Normalized).zip": "ASTI-Spendings",
    "CommodityBalances_Crops_E_All_Data_(Normalized).zip": "Commodity Balances - Crops Primary Equivalent",
    "CommodityBalances_LivestockFish_E_All_Data_(Normalized).zip": "Commodity Balances - Livestock and Fish Primary Equivalent",
    "ConsumerPriceIndices_E_All_Data_(Normalized).zip": "Consumer Price Indices",
    "Deflators_E_All_Data_(Normalized).zip": "Deflators",
    "Development_Assistance_to_Agriculture_E_All_Data_(Normalized).zip": "Development Flows to Agriculture",
    "Emissions_Agriculture_Agriculture_total_E_All_Data_(Normalized).zip": "Agriculture Total",
    "Emissions_Agriculture_Burning_crop_residues_E_All_Data_(Normalized).zip": "Burning - Crop Residues",
    "Emissions_Agriculture_Burning_Savanna_E_All_Data_(Normalized).zip": "Burning - Savanna",
    "Emissions_Agriculture_Crop_Residues_E_All_Data_(Normalized).zip": "Crop Residues",
    "Emissions_Agriculture_Cultivated_Organic_Soils_E_All_Data_(Normalized).zip": "Cultivation of Organic Soils",
    "Emissions_Agriculture_Energy_E_All_Data_(Normalized).zip": "Energy Use",
    "Emissions_Agriculture_Enteric_Fermentation_E_All_Data_(Normalized).zip": "Enteric Fermentation",
    "Emissions_Agriculture_Manure_applied_to_soils_E_All_Data_(Normalized).zip": "Manure applied to Soils",
    "Emissions_Agriculture_Manure_left_on_pasture_E_All_Data_(Normalized).zip": "Manure left on Pasture",
    "Emissions_Agriculture_Manure_Management_E_All_Data_(Normalized).zip": "Manure Management",
    "Emissions_Agriculture_Rice_Cultivation_E_All_Data_(Normalized).zip": "Rice Cultivation",
    "Emissions_Agriculture_Synthetic_Fertilizers_E_All_Data_(Normalized).zip": "Synthetic Fertilizers",
    "Emissions_Land_Use_Burning_Biomass_E_All_Data_(Normalized).zip": "Burning - Biomass",
    "Emissions_Land_Use_Cropland_E_All_Data_(Normalized).zip": "Cropland",
    "Emissions_Land_Use_Forest_Land_E_All_Data_(Normalized).zip": "Forest Land",
    "Emissions_Land_Use_Grassland_E_All_Data_(Normalized).zip": "Grassland",
    "Emissions_Land_Use_Land_Use_Total_E_All_Data_(Normalized).zip": "Land Use Total",
    "Employment_Indicators_E_All_Data_(Normalized).zip": "Employment Indicators",
    "Environment_AirClimateChange_E_All_Data_(Normalized).zip": "Air and climate change",
    "Environment_Emissions_by_Sector_E_All_Data_(Normalized).zip": "Emissions by sector",
    "Environment_Emissions_intensities_E_All_Data_(Normalized).zip": "Emissions intensities",
    "Environment_Temperature_change_E_All_Data_(Normalized).zip": "Temperature change",
    "Environment_Energy_E_All_Data_(Normalized).zip": "Energy",
    "Environment_Fertilizers_E_All_Data_(Normalized).zip": "Fertilizers",
    "Environment_LandCover_E_All_Data_(Normalized).zip": "Land Cover",
    "Environment_LandUse_E_All_Data_(Normalized).zip": "Land Use",
    "Environment_LivestockManure_E_All_Data_(Normalized).zip": "Livestock",
    "Environment_LivestockPatterns_E_All_Data_(Normalized).zip": "Livestock Patterns",
    "Environment_Pesticides_E_All_Data_(Normalized).zip": "Pesticides",
    "Environment_Soil_E_All_Data_(Normalized).zip": "Soil",
    "Environment_Water_E_All_Data_(Normalized).zip": "Water",
    "Exchange_rate_E_All_Data_(Normalized).zip": "Exchange rates - Annual",
    "Food_Aid_Shipments_WFP_E_All_Data_(Normalized).zip": "Food Aid Shipments (WFP)",
    "Food_Security_Data_E_All_Data_(Normalized).zip": "Suite of Food Security Indicators",
    "FoodBalanceSheets_E_All_Data_(Normalized).zip": "Food Balance Sheets",
    "FoodSupply_Crops_E_All_Data_(Normalized).zip": "Food Supply - Crops Primary Equivalent",
    "FoodSupply_LivestockFish_E_All_Data_(Normalized).zip": "Food Supply - Livestock and Fish Primary Equivalent",
    "Forestry_E_All_Data_(Normalized).zip": "Forestry Production and Trade",
    "Forestry_Trade_Flows_E_All_Data_(Normalized).zip": "Forestry Trade Flows",
    "Indicators_from_Household_Surveys_E_All_Data_(Normalized).zip": "Indicators from Household Surveys (gender, area, socioeconomics)",
    "Inputs_FertilizersNutrient_E_All_Data_(Normalized).zip": "Fertilizers Nutrient",
    "Inputs_FertilizersProduct_E_All_Data_(Normalized).zip": "Fertilizers Product",
    "Inputs_FertilizersArchive_E_All_Data_(Normalized).zip": "Fertilizers archive",
    "Inputs_FertilizersTradeValues_E_All_Data_(Normalized).zip": "Fertilizers - Trade Value",
    "Inputs_LandUse_E_All_Data_(Normalized).zip": "Land Use",
    "Inputs_Pesticides_Trade_E_All_Data_(Normalized).zip": "Pesticides Trade",
    "Inputs_Pesticides_Use_E_All_Data_(Normalized).zip": "Pesticides Use",
    "Investment_CapitalStock_E_All_Data_(Normalized).zip": "Capital Stock",
    "Investment_CountryInvestmentStatisticsProfile_E_All_Data_(Normalized).zip": "Country Investment Statistics Profile",
    "Investment_CreditAgriculture_E_All_Data_(Normalized).zip": "Credit to Agriculture",
    "Investment_ForeignDirectInvestment_E_All_Data_(Normalized).zip": "Foreign Direct Investment (FDI)",
    "Investment_GovernmentExpenditure_E_All_Data_(Normalized).zip": "Government Expenditure",
    "Investment_Machinery_E_All_Data_(Normalized).zip": "Machinery",
    "Investment_MachineryArchive_E_All_Data_(Normalized).zip": "Machinery Archive",
    "Macro-Statistics_Key_Indicators_E_All_Data_(Normalized).zip": "Macro Indicators",
    "Population_E_All_Data_(Normalized).zip": "Annual population",
    "Price_Indices_E_All_Data_(Normalized).zip": "Producer Price Indices - Annual",
    "Prices_E_All_Data_(Normalized).zip": "Producer Prices - Annual",
    "Prices_Monthly_E_All_Data_(Normalized).zip": "Producer Prices - Monthly",
    "PricesArchive_E_All_Data_(Normalized).zip": "Producer Prices - Archive",
    "Production_Crops_E_All_Data_(Normalized).zip": "Crops",
    "Production_CropsProcessed_E_All_Data_(Normalized).zip": "Crops processed",
    "Production_Indices_E_All_Data_(Normalized).zip": "Production Indices",
    "Production_Livestock_E_All_Data_(Normalized).zip": "Live Animals",
    "Production_LivestockPrimary_E_All_Data_(Normalized).zip": "Livestock Primary",
    "Production_LivestockProcessed_E_All_Data_(Normalized).zip": "Livestock Processed",
    "Trade_Crops_Livestock_E_All_Data_(Normalized).zip": "Crops and livestock products",
    "Trade_DetailedTradeMatrix_E_All_Data_(Normalized).zip": "Detailed trade matrix",
    "Trade_Indices_E_All_Data_(Normalized).zip": "Trade Indices",
    "Trade_LiveAnimals_E_All_Data_(Normalized).zip": "Live animals",
    "Value_of_Production_E_All_Data_(Normalized).zip": "Value of Agricultural Production",
    "FoodBalanceSheets_E_All_Data_(Normalized).csv": "Food Balance Sheets"
}

In [8]:
new_files = []

for i in tqdm(glob(path_to_data)):
    item = i.split("/")[-1]
    if item not in file_dataset_names:
        new_files.append(item)

100%|██████████| 79/79 [00:00<00:00, 156076.31it/s]


In [9]:
new_files

[]

In [10]:
priority = ["Food_Security_Data_E_All_Data_(Normalized).zip",
"FoodBalanceSheets_E_All_Data_(Normalized).zip",
"FoodSupply_Crops_E_All_Data_(Normalized).zip",
"FoodSupply_LivestockFish_E_All_Data_(Normalized).zip",
"Indicators_from_Household_Surveys_E_All_Data_(Normalized).zip",
"Production_Crops_E_All_Data_(Normalized).zip",
"Production_CropsProcessed_E_All_Data_(Normalized).zip",
"Production_Indices_E_All_Data_(Normalized).zip",
"Production_Livestock_E_All_Data_(Normalized).zip",
"Production_LivestockPrimary_E_All_Data_(Normalized).zip",
"Production_LivestockProcessed_E_All_Data_(Normalized).zip",         
"CommodityBalances_Crops_E_All_Data_(Normalized).zip",
"CommodityBalances_LivestockFish_E_All_Data_(Normalized).zip",
"Emissions_Agriculture_Agriculture_total_E_All_Data_(Normalized).zip",
"Emissions_Agriculture_Burning_crop_residues_E_All_Data_(Normalized).zip",
"Emissions_Agriculture_Burning_Savanna_E_All_Data_(Normalized).zip",         
"Emissions_Agriculture_Crop_Residues_E_All_Data_(Normalized).zip",
"Emissions_Agriculture_Cultivated_Organic_Soils_E_All_Data_(Normalized).zip",
"Emissions_Agriculture_Energy_E_All_Data_(Normalized).zip",
"Emissions_Agriculture_Enteric_Fermentation_E_All_Data_(Normalized).zip",
"Emissions_Agriculture_Manure_applied_to_soils_E_All_Data_(Normalized).zip",        
"Emissions_Agriculture_Manure_left_on_pasture_E_All_Data_(Normalized).zip",
"Emissions_Agriculture_Manure_Management_E_All_Data_(Normalized).zip",
"Emissions_Agriculture_Rice_Cultivation_E_All_Data_(Normalized).zip",
"Emissions_Agriculture_Synthetic_Fertilizers_E_All_Data_(Normalized).zip",
"Emissions_Land_Use_Burning_Biomass_E_All_Data_(Normalized).zip",          
"Emissions_Land_Use_Cropland_E_All_Data_(Normalized).zip",
"Emissions_Land_Use_Forest_Land_E_All_Data_(Normalized).zip",
"Emissions_Land_Use_Grassland_E_All_Data_(Normalized).zip",
"Emissions_Land_Use_Land_Use_Total_E_All_Data_(Normalized).zip",
"Employment_Indicators_E_All_Data_(Normalized).zip",       
"Environment_Emissions_by_Sector_E_All_Data_(Normalized).zip",
"Environment_Emissions_intensities_E_All_Data_(Normalized).zip",
"Environment_Fertilizers_E_All_Data_(Normalized).zip",
"Environment_LandCover_E_All_Data_(Normalized).zip",
"Environment_LandUse_E_All_Data_(Normalized).zip",      
"Inputs_Pesticides_Use_E_All_Data_(Normalized).zip",          
"ASTI_Expenditures_E_All_Data_(Normalized).zip",
"ASTI_Researchers_E_All_Data_(Normalized).zip",          
"ConsumerPriceIndices_E_All_Data_(Normalized).zip",
"Deflators_E_All_Data_(Normalized).zip",
"Development_Assistance_to_Agriculture_E_All_Data_(Normalized).zip",
"Environment_LivestockPatterns_E_All_Data_(Normalized).zip",
"Environment_Pesticides_E_All_Data_(Normalized).zip",
"Exchange_rate_E_All_Data_(Normalized).zip",
"Food_Aid_Shipments_WFP_E_All_Data_(Normalized).zip",
"Forestry_E_All_Data_(Normalized).zip",
"Forestry_Trade_Flows_E_All_Data_(Normalized).zip",
"Inputs_FertilizersArchive_E_All_Data_(Normalized).zip",         
"Inputs_FertilizersNutrient_E_All_Data_(Normalized).zip",
"Inputs_FertilizersProduct_E_All_Data_(Normalized).zip",           
"Environment_Temperature_change_E_All_Data_(Normalized).zip",
"Environment_LivestockManure_E_All_Data_(Normalized).zip", 
"Inputs_LandUse_E_All_Data_(Normalized).zip",
"Inputs_Pesticides_Trade_E_All_Data_(Normalized).zip",
"Investment_CapitalStock_E_All_Data_(Normalized).zip",
"Investment_CountryInvestmentStatisticsProfile_E_All_Data_(Normalized).zip",   
"Investment_CreditAgriculture_E_All_Data_(Normalized).zip",
"Investment_ForeignDirectInvestment_E_All_Data_(Normalized).zip",
"Investment_GovernmentExpenditure_E_All_Data_(Normalized).zip",
"Investment_Machinery_E_All_Data_(Normalized).zip",           
"Investment_MachineryArchive_E_All_Data_(Normalized).zip",       
"Macro-Statistics_Key_Indicators_E_All_Data_(Normalized).zip",
"Population_E_All_Data_(Normalized).zip",
"Price_Indices_E_All_Data_(Normalized).zip",
"Prices_E_All_Data_(Normalized).zip",
"Prices_Monthly_E_All_Data_(Normalized).zip",
"PricesArchive_E_All_Data_(Normalized).zip",    
"Trade_Crops_Livestock_E_All_Data_(Normalized).zip",
"Trade_DetailedTradeMatrix_E_All_Data_(Normalized).zip",
"Trade_Indices_E_All_Data_(Normalized).zip",
"Trade_LiveAnimals_E_All_Data_(Normalized).zip",
"Value_of_Production_E_All_Data_(Normalized).zip",     
"ASTI_Research_Spending_E_All_Data_(Normalized).zip",
"Environment_AirClimateChange_E_All_Data_(Normalized).zip",
"Environment_Energy_E_All_Data_(Normalized).zip",      
"Environment_Soil_E_All_Data_(Normalized).zip",
"Environment_Water_E_All_Data_(Normalized).zip",
"Inputs_FertilizersTradeValues_E_All_Data_(Normalized).zip"]

## Datasets

In [11]:
names = []

for x in glob(path_to_data):
    x = x.split("/")[-1]
    
    found = False
  
    if x in priority:
        for j in category_files:
            if x in category_files[j]:
                names.append(j + ": " + file_dataset_names[x] + " - FAO (2019)")
                found = True
        if not found:
            names.append(file_dataset_names[x] + " - FAO (2019)")
datasets = pd.DataFrame()
datasets['name'] = names
datasets['id'] = [x for x in range(len(names))]
datasets.to_csv("datasets.csv", index=False)

In [12]:
datasets['name'].values

array(['Forestry: Forestry Production and Trade - FAO (2019)',
       'Macro-Statistics: Macro Indicators - FAO (2019)',
       'Production: Livestock Primary - FAO (2019)',
       'Crop Residues - FAO (2019)', 'Enteric Fermentation - FAO (2019)',
       'Investment: Country Investment Statistics Profile - FAO (2019)',
       'Cropland - FAO (2019)',
       'Emissions - Agriculture: Burning - Savanna - FAO (2019)',
       'Inputs: Pesticides Use - FAO (2019)',
       'Production: Crops processed - FAO (2019)',
       'Agri-Environmental Indicators: Air and climate change - FAO (2019)',
       'Machinery - FAO (2019)',
       'Agri-Environmental Indicators: Soil - FAO (2019)',
       'Prices: Exchange rates - Annual - FAO (2019)',
       'Producer Prices - Archive - FAO (2019)',
       'Trade: Crops and livestock products - FAO (2019)',
       'Forest Land - FAO (2019)',
       'Production: Livestock Processed - FAO (2019)',
       'Agri-Environmental Indicators: Fertilizers - FAO (2019

## Sources

In [17]:
# We need this info to populate additionalInfo and dataPublisherSource in database (sources table).
# This info was collected manually and it takes time, not sure if we need it.


names_publisher_source = {
        "Production: Livestock Primary - FAO (2019)": get_pub_source(metadata_by_name["Livestock Primary"]),
        "Production: Crops processed - FAO (2019)": get_pub_source(metadata_by_name["Crops processed"]),
        "Production: Livestock Processed - FAO (2019)": get_pub_source(metadata_by_name["Livestock Processed"]),
        "Food Balance: Food Supply - Livestock and Fish Primary Equivalent - FAO (2019)": get_pub_source(metadata_by_name["Food Supply - Livestock and Fish Primary Equivalent"]),
        "Production: Crops - FAO (2019)": get_pub_source(metadata_by_name["Crops"]),
        "Food Security: Indicators from Household Surveys (gender, area, socioeconomics) - FAO (2019)": get_pub_source(metadata_by_name["Suite of Food Security Indicators"]),
        "Production: Production Indices - FAO (2019)": get_pub_source(metadata_by_name["Production Indices"]),
        "Production: Live Animals - FAO (2019)": get_pub_source(metadata_by_name["Live Animals"]),
        "Food Balance: Food Balance Sheets - FAO (2019)": get_pub_source(metadata_by_name["Food Balances (old methodology and population)"]),
        "Food Balance: Food Supply - Crops Primary Equivalent - FAO (2019)": get_pub_source(metadata_by_name["Food Supply - Crops Primary Equivalent"]),
        "Food Security: Suite of Food Security Indicators - FAO (2019)": get_pub_source(metadata_by_name["Suite of Food Security Indicators"]),
        "Food Balance: Commodity Balances - Crops Primary Equivalent - FAO (2019)": get_pub_source(metadata_by_name["Commodity Balances - Crops Primary Equivalent"]),
        "Food Balance: Commodity Balances - Livestock and Fish Primary Equivalent - FAO (2019)": get_pub_source(metadata_by_name["Commodity Balances - Livestock and Fish Primary Equivalent"]),
        "Emissions - Agriculture: Agriculture Total - FAO (2019)": get_pub_source(metadata_by_name["Agriculture Total"]),
        "Burning - Crop Residues - FAO (2019)": get_pub_source(metadata_by_name["Burning - Crop Residues"]),
        "Emissions - Agriculture: Burning - Savanna - FAO (2019)": get_pub_source(metadata_by_name["Burning - Savanna"]),
        "Crop Residues - FAO (2019)": get_pub_source(metadata_by_name["Crop Residues"]),
        "Cultivation of Organic Soils - FAO (2019)": get_pub_source(metadata_by_name["Cultivation of Organic Soils"]),
        "Energy Use - FAO (2019)": get_pub_source(metadata_by_name["Energy Use"]),
        "Enteric Fermentation - FAO (2019)": get_pub_source(metadata_by_name["Enteric Fermentation"]),
        "Manure applied to Soils - FAO (2019)": get_pub_source(metadata_by_name["Manure applied to Soils"]),
        "Manure left on Pasture - FAO (2019)": get_pub_source(metadata_by_name["Manure left on Pasture"]),
        "Manure Management - FAO (2019)": get_pub_source(metadata_by_name["Manure Management"]),
        "Rice Cultivation - FAO (2019)": get_pub_source(metadata_by_name["Rice Cultivation"]),
        "Synthetic Fertilizers - FAO (2019)": get_pub_source(metadata_by_name["Synthetic Fertilizers"]),
        "Burning - Biomass - FAO (2019)": get_pub_source(metadata_by_name["Burning - Biomass"]),
        "Cropland - FAO (2019)": get_pub_source(metadata_by_name["Cropland"]),
        "Forest Land - FAO (2019)": get_pub_source(metadata_by_name["Forest Land"]),
        "Grassland - FAO (2019)": get_pub_source(metadata_by_name["Grassland"]),
        "Land Use Total - FAO (2019)": get_pub_source(metadata_by_name["Land Use Total"]),
        "Employment Indicators - FAO (2019)": get_pub_source(metadata_by_name["Employment Indicators"]),
        "Agri-Environmental Indicators: Emissions by sector - FAO (2019)": get_pub_source(metadata_by_name["Emissions by sector"]),
        "Agri-Environmental Indicators: Emissions intensities - FAO (2019)": get_pub_source(metadata_by_name["Emissions intensities"]),
        "Agri-Environmental Indicators: Fertilizers - FAO (2019)": get_pub_source(metadata_by_name["Fertilizers indicators"]),
        "Agri-Environmental Indicators: Land Cover - FAO (2019)": get_pub_source(metadata_by_name["Land Cover"]),
        "Agri-Environmental Indicators: Land Use - FAO (2019)": get_pub_source(metadata_by_name["Land Use"]),
        "Inputs: Pesticides Use - FAO (2019)": get_pub_source(metadata_by_name["Pesticides Use"]),
        "ASTI R&D Indicators: ASTI-Expenditures - FAO (2019)": get_pub_source(metadata_by_name["ASTI-Expenditures"]),
        "ASTI R&D Indicators: ASTI-Researchers - FAO (2019)": get_pub_source(metadata_by_name["ASTI-Researchers"]),
        "Prices: Consumer Price Indices - FAO (2019)": get_pub_source(metadata_by_name["Consumer Price Indices"]),
        "Prices: Deflators - FAO (2019)": get_pub_source(metadata_by_name["Deflators"]),
        "Investment: Development Flows to Agriculture - FAO (2019)": get_pub_source(metadata_by_name["Development Flows to Agriculture"]),
        "Agri-Environmental Indicators: Livestock Patterns - FAO (2019)": get_pub_source(metadata_by_name["Livestock Patterns"]),
        "Agri-Environmental Indicators: Pesticides - FAO (2019)": get_pub_source(metadata_by_name["Pesticides indicators"]),
        "Prices: Exchange rates - Annual - FAO (2019)": get_pub_source(metadata_by_name["Exchange rates - Annual"]),
        "Emergency Response: Food Aid Shipments (WFP) - FAO (2019)": get_pub_source(metadata_by_name["Food Aid Shipments (WFP)"]),
        "Forestry: Forestry Production and Trade - FAO (2019)": get_pub_source(metadata_by_name["Forestry Production and Trade"]),
        "Forestry: Forestry Trade Flows - FAO (2019)": get_pub_source(metadata_by_name["Forestry Trade Flows"]),
        "Fertilizers archive - FAO (2019)": get_pub_source(metadata_by_name["Fertilizers archive"]),
        "Inputs: Fertilizers Nutrient - FAO (2019)": get_pub_source(metadata_by_name["Fertilizers by Nutrient"]),
        "Inputs: Fertilizers Product - FAO (2019)": get_pub_source(metadata_by_name["Fertilizers by Product"]),
        "Agri-Environmental Indicators: Temperature change - FAO (2019)": get_pub_source(metadata_by_name["Temperature change"]),
        "Agri-Environmental Indicators: Livestock - FAO (2019)": get_pub_source(metadata_by_name["Livestock Manure"]),
        "Inputs: Land Use - FAO (2019)": get_pub_source(metadata_by_name["Land Use"]),
        "Pesticides Trade - FAO (2019)": get_pub_source(metadata_by_name["Pesticides Trade"]),
        "Macro-Statistics: Capital Stock - FAO (2019)": get_pub_source(metadata_by_name["Capital Stock"]),
        "Investment: Country Investment Statistics Profile - FAO (2019)": get_pub_source(metadata_by_name["Country Investment Statistics Profile"]),
        "Investment: Credit to Agriculture - FAO (2019)": get_pub_source(metadata_by_name["Credit to Agriculture"]),
        "Foreign Direct Investment (FDI) - FAO (2019)": get_pub_source(metadata_by_name["Foreign Direct Investment (FDI)"]),
        "Investment: Government Expenditure - FAO (2019)": get_pub_source(metadata_by_name["Government Expenditure"]),
        "Machinery - FAO (2019)": get_pub_source(metadata_by_name["Machinery"]),
        "Machinery Archive - FAO (2019)": get_pub_source(metadata_by_name["Machinery Archive"]),
        "Macro-Statistics: Macro Indicators - FAO (2019)": get_pub_source(metadata_by_name["Macro Indicators"]),
        "Annual population - FAO (2019)": get_pub_source(metadata_by_name["Annual population"]),
        "Prices: Producer Price Indices - Annual - FAO (2019)": get_pub_source(metadata_by_name["Producer Price Indices - Annual"]),
        "Prices: Producer Prices - Annual - FAO (2019)": get_pub_source(metadata_by_name["Producer Prices - Annual"]),
        "Prices: Producer Prices - Monthly - FAO (2019)": get_pub_source(metadata_by_name["Producer Prices - Monthly"]),
        "Producer Prices - Archive - FAO (2019)": get_pub_source(metadata_by_name["Producer Prices - Archive"]),
        "Trade: Crops and livestock products - FAO (2019)": get_pub_source(metadata_by_name["Crops and livestock products"]),
        "Detailed trade matrix - FAO (2019)": get_pub_source(metadata_by_name["Detailed trade matrix"]),
        "Trade Indices - FAO (2019)": get_pub_source(metadata_by_name["Trade Indices"]),
        "Trade: Live animals - FAO (2019)": get_pub_source(metadata_by_name["Live animals"]),
        "Production: Value of Agricultural Production - FAO (2019)": get_pub_source(metadata_by_name["Value of Agricultural Production"]),
        "ASTI R&D Indicators: ASTI-Spendings - FAO (2019)": "",
        "Agri-Environmental Indicators: Air and climate change - FAO (2019)": "",
        "Agri-Environmental Indicators: Energy - FAO (2019)": "",
        "Agri-Environmental Indicators: Soil - FAO (2019)": "",
        "Agri-Environmental Indicators: Water - FAO (2019)": "",
        "Inputs: Fertilizers - Trade Value - FAO (2019)": ""
                         }
names_add_info_source = {
        "Production: Livestock Primary - FAO (2019)": get_add_info(metadata_by_name["Livestock Primary"]),
        "Production: Crops processed - FAO (2019)": get_add_info(metadata_by_name["Crops processed"]),
        "Production: Livestock Processed - FAO (2019)": get_add_info(metadata_by_name["Livestock Processed"]),
        "Food Balance: Food Supply - Livestock and Fish Primary Equivalent - FAO (2019)": get_add_info(metadata_by_name["Food Supply - Livestock and Fish Primary Equivalent"]),
        "Production: Crops - FAO (2019)": get_add_info(metadata_by_name["Crops"]),
        "Food Security: Indicators from Household Surveys (gender, area, socioeconomics) - FAO (2019)": get_add_info(metadata_by_name["Suite of Food Security Indicators"]),
        "Production: Production Indices - FAO (2019)": get_add_info(metadata_by_name["Production Indices"]),
        "Production: Live Animals - FAO (2019)": get_add_info(metadata_by_name["Live Animals"]),
        "Food Balance: Food Balance Sheets - FAO (2019)": get_add_info(metadata_by_name["Food Balances (old methodology and population)"]),
        "Food Balance: Food Supply - Crops Primary Equivalent - FAO (2019)": get_add_info(metadata_by_name["Food Supply - Crops Primary Equivalent"]),
        "Food Security: Suite of Food Security Indicators - FAO (2019)": get_add_info(metadata_by_name["Suite of Food Security Indicators"]),
        "Food Balance: Commodity Balances - Crops Primary Equivalent - FAO (2019)": get_add_info(metadata_by_name["Commodity Balances - Crops Primary Equivalent"]),
        "Food Balance: Commodity Balances - Livestock and Fish Primary Equivalent - FAO (2019)": get_add_info(metadata_by_name["Commodity Balances - Livestock and Fish Primary Equivalent"]),
        "Emissions - Agriculture: Agriculture Total - FAO (2019)": get_add_info(metadata_by_name["Agriculture Total"]),
        "Burning - Crop Residues - FAO (2019)": get_add_info(metadata_by_name["Burning - Crop Residues"]),
        "Emissions - Agriculture: Burning - Savanna - FAO (2019)": get_add_info(metadata_by_name["Burning - Savanna"]),
        "Crop Residues - FAO (2019)": get_add_info(metadata_by_name["Crop Residues"]),
        "Cultivation of Organic Soils - FAO (2019)": get_add_info(metadata_by_name["Cultivation of Organic Soils"]),
        "Energy Use - FAO (2019)": get_add_info(metadata_by_name["Energy Use"]),
        "Enteric Fermentation - FAO (2019)": get_add_info(metadata_by_name["Enteric Fermentation"]),
        "Manure applied to Soils - FAO (2019)": get_add_info(metadata_by_name["Manure applied to Soils"]),
        "Manure left on Pasture - FAO (2019)": get_add_info(metadata_by_name["Manure left on Pasture"]),
        "Manure Management - FAO (2019)": get_add_info(metadata_by_name["Manure Management"]),
        "Rice Cultivation - FAO (2019)": get_add_info(metadata_by_name["Rice Cultivation"]),
        "Synthetic Fertilizers - FAO (2019)": get_add_info(metadata_by_name["Synthetic Fertilizers"]),
        "Burning - Biomass - FAO (2019)": get_add_info(metadata_by_name["Burning - Biomass"]),
        "Cropland - FAO (2019)": get_add_info(metadata_by_name["Cropland"]),
        "Forest Land - FAO (2019)": get_add_info(metadata_by_name["Forest Land"]),
        "Grassland - FAO (2019)": get_add_info(metadata_by_name["Grassland"]),
        "Land Use Total - FAO (2019)": get_add_info(metadata_by_name["Land Use Total"]),
        "Employment Indicators - FAO (2019)": get_add_info(metadata_by_name["Employment Indicators"]),
        "Agri-Environmental Indicators: Emissions by sector - FAO (2019)": get_add_info(metadata_by_name["Emissions by sector"]),
        "Agri-Environmental Indicators: Emissions intensities - FAO (2019)": get_add_info(metadata_by_name["Emissions intensities"]),
        "Agri-Environmental Indicators: Fertilizers - FAO (2019)": get_add_info(metadata_by_name["Fertilizers indicators"]),
        "Agri-Environmental Indicators: Land Cover - FAO (2019)": get_add_info(metadata_by_name["Land Cover"]),
        "Agri-Environmental Indicators: Land Use - FAO (2019)": get_add_info(metadata_by_name["Land Use"]),
        "Inputs: Pesticides Use - FAO (2019)": get_add_info(metadata_by_name["Pesticides Use"]),
        "ASTI R&D Indicators: ASTI-Expenditures - FAO (2019)": get_add_info(metadata_by_name["ASTI-Expenditures"]),
        "ASTI R&D Indicators: ASTI-Researchers - FAO (2019)": get_add_info(metadata_by_name["ASTI-Researchers"]),
        "Prices: Consumer Price Indices - FAO (2019)": get_add_info(metadata_by_name["Consumer Price Indices"]),
        "Prices: Deflators - FAO (2019)": get_add_info(metadata_by_name["Deflators"]),
        "Investment: Development Flows to Agriculture - FAO (2019)": get_add_info(metadata_by_name["Development Flows to Agriculture"]),
        "Agri-Environmental Indicators: Livestock Patterns - FAO (2019)": get_add_info(metadata_by_name["Livestock Patterns"]),
        "Agri-Environmental Indicators: Pesticides - FAO (2019)": get_add_info(metadata_by_name["Pesticides indicators"]),
        "Prices: Exchange rates - Annual - FAO (2019)": get_add_info(metadata_by_name["Exchange rates - Annual"]),
        "Emergency Response: Food Aid Shipments (WFP) - FAO (2019)": get_add_info(metadata_by_name["Food Aid Shipments (WFP)"]),
        "Forestry: Forestry Production and Trade - FAO (2019)": get_add_info(metadata_by_name["Forestry Production and Trade"]),
        "Forestry: Forestry Trade Flows - FAO (2019)": get_add_info(metadata_by_name["Forestry Trade Flows"]),
        "Fertilizers archive - FAO (2019)": get_add_info(metadata_by_name["Fertilizers archive"]),
        "Inputs: Fertilizers Nutrient - FAO (2019)": get_add_info(metadata_by_name["Fertilizers by Nutrient"]),
        "Inputs: Fertilizers Product - FAO (2019)": get_add_info(metadata_by_name["Fertilizers by Product"]),
        "Agri-Environmental Indicators: Temperature change - FAO (2019)": get_add_info(metadata_by_name["Temperature change"]),
        "Agri-Environmental Indicators: Livestock - FAO (2019)": get_add_info(metadata_by_name["Livestock Manure"]),
        "Inputs: Land Use - FAO (2019)": get_add_info(metadata_by_name["Land Use"]),
        "Pesticides Trade - FAO (2019)": get_add_info(metadata_by_name["Pesticides Trade"]),
        "Macro-Statistics: Capital Stock - FAO (2019)": get_add_info(metadata_by_name["Capital Stock"]),
        "Investment: Country Investment Statistics Profile - FAO (2019)": get_add_info(metadata_by_name["Country Investment Statistics Profile"]),
        "Investment: Credit to Agriculture - FAO (2019)": get_add_info(metadata_by_name["Credit to Agriculture"]),
        "Foreign Direct Investment (FDI) - FAO (2019)": get_add_info(metadata_by_name["Foreign Direct Investment (FDI)"]),
        "Investment: Government Expenditure - FAO (2019)": get_add_info(metadata_by_name["Government Expenditure"]),
        "Machinery - FAO (2019)": get_add_info(metadata_by_name["Machinery"]),
        "Machinery Archive - FAO (2019)": get_add_info(metadata_by_name["Machinery Archive"]),
        "Macro-Statistics: Macro Indicators - FAO (2019)": get_add_info(metadata_by_name["Macro Indicators"]),
        "Annual population - FAO (2019)": get_add_info(metadata_by_name["Annual population"]),
        "Prices: Producer Price Indices - Annual - FAO (2019)": get_add_info(metadata_by_name["Producer Price Indices - Annual"]),
        "Prices: Producer Prices - Annual - FAO (2019)": get_add_info(metadata_by_name["Producer Prices - Annual"]),
        "Prices: Producer Prices - Monthly - FAO (2019)": get_add_info(metadata_by_name["Producer Prices - Monthly"]),
        "Producer Prices - Archive - FAO (2019)": get_add_info(metadata_by_name["Producer Prices - Archive"]),
        "Trade: Crops and livestock products - FAO (2019)": get_add_info(metadata_by_name["Crops and livestock products"]),
        "Detailed trade matrix - FAO (2019)": get_add_info(metadata_by_name["Detailed trade matrix"]),
        "Trade Indices - FAO (2019)": get_add_info(metadata_by_name["Trade Indices"]),
        "Trade: Live animals - FAO (2019)": get_add_info(metadata_by_name["Live animals"]),
        "Production: Value of Agricultural Production - FAO (2019)": get_add_info(metadata_by_name["Value of Agricultural Production"]),
        "ASTI R&D Indicators: ASTI-Spendings - FAO (2019)": "",
        "Agri-Environmental Indicators: Air and climate change - FAO (2019)": "",
        "Agri-Environmental Indicators: Energy - FAO (2019)": "",
        "Agri-Environmental Indicators: Soil - FAO (2019)": "",
        "Agri-Environmental Indicators: Water - FAO (2019)": "",
        "Inputs: Fertilizers - Trade Value - FAO (2019)": ""
}

files_to_description = {
    
        "Food_Security_Data_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Suite of Food Security Indicators"]),
        "FoodBalanceSheets_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Food Balances (old methodology and population)"]),
        "FoodSupply_Crops_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Food Supply - Crops Primary Equivalent"]),
        "FoodSupply_LivestockFish_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Food Supply - Livestock and Fish Primary Equivalent"]),
        "Indicators_from_Household_Surveys_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Suite of Food Security Indicators"]),
        "Production_Crops_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Crops"]),
        "Production_CropsProcessed_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Crops processed"]), 
        "Production_Indices_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Production Indices"]), 
        "Production_Livestock_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Live Animals"]), 
        "Production_LivestockPrimary_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Livestock Primary"]), 
        "Production_LivestockProcessed_E_All_Data_(Normalized).zip":get_pub_desc(metadata_by_name["Livestock Processed"]),  
        "CommodityBalances_Crops_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Commodity Balances - Crops Primary Equivalent"]),
        "CommodityBalances_LivestockFish_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Commodity Balances - Livestock and Fish Primary Equivalent"]),
        "Emissions_Agriculture_Agriculture_total_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Agriculture Total"]),
        "Emissions_Agriculture_Burning_crop_residues_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Burning - Crop Residues"]),
        "Emissions_Agriculture_Burning_Savanna_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Burning - Savanna"]),
        "Emissions_Agriculture_Crop_Residues_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Crop Residues"]),
        "Emissions_Agriculture_Cultivated_Organic_Soils_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Cultivation of Organic Soils"]),
        "Emissions_Agriculture_Energy_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Energy Use"]),
        "Emissions_Agriculture_Enteric_Fermentation_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Enteric Fermentation"]),
        "Emissions_Agriculture_Manure_applied_to_soils_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Manure applied to Soils"]),
        "Emissions_Agriculture_Manure_left_on_pasture_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Manure left on Pasture"]),
        "Emissions_Agriculture_Manure_Management_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Manure Management"]),
        "Emissions_Agriculture_Rice_Cultivation_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Rice Cultivation"]),
        "Emissions_Agriculture_Synthetic_Fertilizers_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Synthetic Fertilizers"]),
        "Emissions_Land_Use_Burning_Biomass_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Burning - Biomass"]),
        "Emissions_Land_Use_Cropland_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Cropland"]),
        "Emissions_Land_Use_Forest_Land_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Forest Land"]),
        "Emissions_Land_Use_Grassland_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Grassland"]),
        "Emissions_Land_Use_Land_Use_Total_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Land Use Total"]),
        "Employment_Indicators_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Employment Indicators"]),
        "Environment_Emissions_by_Sector_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Emissions by sector"]),
        "Environment_Emissions_intensities_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Emissions intensities"]),
        "Environment_Fertilizers_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Fertilizers indicators"]),
        "Environment_LandCover_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Land Cover"]),
        "Environment_LandUse_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Land Use"]),
        "Inputs_Pesticides_Use_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Pesticides Use"]),
        "ASTI_Expenditures_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["ASTI-Expenditures"]),
        "ASTI_Researchers_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["ASTI-Researchers"]),
        "ConsumerPriceIndices_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Consumer Price Indices"]),
        "Deflators_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Deflators"]),
        "Development_Assistance_to_Agriculture_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Development Flows to Agriculture"]),
        "Environment_LivestockPatterns_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Livestock Patterns"]),
        "Environment_Pesticides_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Pesticides indicators"]),
        "Exchange_rate_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Exchange rates - Annual"]),
        "Food_Aid_Shipments_WFP_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Food Aid Shipments (WFP)"]),
        "Forestry_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Forestry Production and Trade"]),
        "Forestry_Trade_Flows_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Forestry Trade Flows"]),
        "Inputs_FertilizersArchive_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Fertilizers archive"]),
        "Inputs_FertilizersNutrient_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Fertilizers by Nutrient"]),
        "Inputs_FertilizersProduct_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Fertilizers by Product"]),
        "Environment_Temperature_change_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Temperature change"]),
        "Environment_LivestockManure_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Livestock Manure"]),
        "Inputs_LandUse_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Land Use"]),
        "Inputs_Pesticides_Trade_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Pesticides Trade"]),
        "Investment_CapitalStock_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Capital Stock"]),
        "Investment_CountryInvestmentStatisticsProfile_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Country Investment Statistics Profile"]),
        "Investment_CreditAgriculture_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Credit to Agriculture"]),
        "Investment_ForeignDirectInvestment_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Foreign Direct Investment (FDI)"]),
        "Investment_GovernmentExpenditure_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Government Expenditure"]),
        "Investment_Machinery_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Machinery"]),           
        "Investment_MachineryArchive_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Machinery Archive"]),
        "Macro-Statistics_Key_Indicators_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Macro Indicators"]),
        "Population_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Annual population"]),
        "Price_Indices_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Producer Price Indices - Annual"]),
        "Prices_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Producer Prices - Annual"]),
        "Prices_Monthly_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Producer Prices - Monthly"]),
        "PricesArchive_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Producer Prices - Archive"]),
        "Trade_Crops_Livestock_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Crops and livestock products"]),
        "Trade_DetailedTradeMatrix_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Detailed trade matrix"]),
        "Trade_Indices_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Trade Indices"]),
        "Trade_LiveAnimals_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Live animals"]),
        "Value_of_Production_E_All_Data_(Normalized).zip": get_pub_desc(metadata_by_name["Value of Agricultural Production"]),
        "ASTI_Research_Spending_E_All_Data_(Normalized).zip": "",
        "Environment_AirClimateChange_E_All_Data_(Normalized).zip": "",
        "Environment_Energy_E_All_Data_(Normalized).zip": "",
        "Environment_Soil_E_All_Data_(Normalized).zip": "",
        "Environment_Water_E_All_Data_(Normalized).zip": "",
        "Inputs_FertilizersTradeValues_E_All_Data_(Normalized).zip": ""
}

In [18]:
names, desc, d_ids = [], [], []


for i, row in datasets.iterrows():
    description = {}
    description['dataPublishedBy'] = "Food and Agriculture Organization of the United Nations (FAO) (2019) "
    description['dataPublisherSource'] = names_publisher_source[row['name']]
    description['link'] = "http://www.fao.org/faostat/en/?#data/"
    description['retrievedDate'] = datetime.datetime.now().strftime("%d-%b-%Y")
    description['additionalInfo'] = names_add_info_source[row['name']]
    dataset_id = row['id']
    
    source_name = row['name'].split(":")[-1]

    names.append(source_name)
    desc.append(json.dumps(description))
    d_ids.append(dataset_id)
    
sources = pd.DataFrame()
sources['name'] = names
sources['description'] = desc
sources['dataset_id'] = d_ids
sources.to_csv("sources.csv", index=False)

## Variables

In [19]:
import csv

ids, names, units, dataset_ids = [], [], [], []

varname_description = {} # it's for populate variables description in the database while inserting in db

unique_var_names = []

ind = 0
for item in tqdm(glob(path_to_data)):
    item_name = item.split("/")[-1] 
    if item_name in priority:
    
        prefix = file_dataset_names[item_name]
        zip_ref = zipfile.ZipFile(item, 'r')
        if item_name == "Investment_CountryInvestmentStatisticsProfile_E_All_Data_(Normalized).zip":
            csv_filename = zip_ref.namelist()[1]
        else:
            csv_filename = zip_ref.namelist()[0]
        zip_ref.extract(csv_filename, path_to_unrar)
        zip_ref.close()
        
        for j in category_files:
            if item_name in category_files[j]:
                dataset_name = j + ": " + file_dataset_names[item_name] + " - FAO (2019)"
                dataset_id = datasets[datasets['name'] == dataset_name]['id'].values[0]
                
                
    
        
        data = pd.read_csv(path_to_unrar + csv_filename, encoding='latin-1')
        
        filecolumns = tuple(data.columns)
        if filecolumns != column_types[8] and filecolumns != column_types[9] and filecolumns != column_types[10] \
                and filecolumns != column_types[11] and filecolumns != column_types[5] and filecolumns != column_types[12]:
            data = data.drop_duplicates(subset=['Item', 'Element', 'Unit'])[['Item', 'Element', 'Unit']]
        elif filecolumns == column_types[9]:
            data = data.drop_duplicates(subset=['Indicator', 'Unit'])[['Indicator', 'Unit']]
        elif filecolumns == column_types[10] or filecolumns == column_types[11] or filecolumns == column_types[5]:
            data = data.drop_duplicates(subset=['Item', 'Unit'])[['Item', 'Unit']]
        elif filecolumns == column_types[12]:
            data = data.drop_duplicates(subset=['Element', 'Unit'])[['Element', 'Unit']]
        else:
            data = data.drop_duplicates(subset=['Breakdown Variable', 'Unit'])[['Breakdown Variable', 'Unit']]

        

        if filecolumns == column_types[0] or filecolumns == column_types[1] \
           or filecolumns == column_types[2] or filecolumns == column_types[3] \
           or filecolumns == column_types[4] or filecolumns == column_types[5] or filecolumns == column_types[8] or filecolumns == column_types[9] \
            or filecolumns == column_types[10] or filecolumns == column_types[11] or filecolumns == column_types[12]:
            
            for i,row in data.iterrows():
                if filecolumns == column_types[0] or filecolumns == column_types[5]:
                    variablename = row['Item']                    

                if filecolumns == column_types[1]:
                    variablename = '%s - %s' % (row['Item'], row['Element'])
                    
                    
                if filecolumns == column_types[2]:
                    variablename = '%s - %s' % (row['Item'], row['Element'])
                if filecolumns == column_types[3]:
                    variablename = '%s - %s' % (row['Item'], row['Element'])
                if filecolumns == column_types[4]:
                    variablename = '%s - %s' % (row['Indicator'], row['Source'])
                    
                if filecolumns == column_types[8]:
                    variablename = row['Breakdown Variable'] 
                if filecolumns == column_types[9]:
                    variablename = row['Indicator']
                if filecolumns == column_types[10] or filecolumns == column_types[11] or filecolumns == column_types[5]:
                    variablename = row['Item']
                if filecolumns == column_types[12]:
                    variablename = row['Element']
                    
                if item_name == 'Emissions_Agriculture_Energy_E_All_Data_(Normalized).zip' or \
                item_name == 'Production_LivestockPrimary_E_All_Data_(Normalized).zip' or item_name == 'Trade_LiveAnimals_E_All_Data_(Normalized).zip':
                    
                    variablename += ' - %s' % row['Unit']
                    
                variablename += " (FAO (2019))"   
                    
                    
                    
                
                    
                

                if variablename not in unique_var_names:
                    varname_description[variablename] = files_to_description[item_name]
                    unique_var_names.append(variablename)
                    units.append(row['Unit'])
                    ids.append(ind)
                    ind+=1
                    dataset_ids.append(dataset_id)
        

  1%|▏         | 1/79 [00:02<02:52,  2.21s/it]/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 52%|█████▏    | 41/79 [00:40<00:42,  1.11s/it]/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
 61%|██████    | 48/79 [01:03<00:31,  1.01s/it]/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
100%|██████████| 79/79 [02:43<00:00,  2.06s/it]


In [20]:
variables = pd.DataFrame()
variables['id'] = ids
variables['name'] = unique_var_names
variables['unit'] = units
variables['dataset_id'] = dataset_ids

In [21]:
variables.to_csv("variables.csv", index=False)

## Datapoints

This functions is for creating variables for each file. As you can see, we created variables multiple ways according to the file type. Here we create a column containing these variables, then get unique and for each unique we get subset of the original data. It works quite long for a large file (1gb<).

In [84]:
def create_var_val_1_2(row):
    
    year = str(row['Year']).split("-")
    
    if len(year) > 1:
        try:
            row['Year'] = (int(str(row['Year']).split("-")[0]) + (int(str(row['Year']).split("-")[1])))//2
        except:
            print(row['Year'])
    else:
        row['Year'] = int(row['Year'])
    
    
    return row


def create_var_val_8(row):
    
    year = row['Survey'].split()[-1]
    
    if len(year) > 4:
        try:
            row['Year'] = (int(row['Survey'].split()[-1].split("-")[0]) + (int(row['Survey'].split()[-1].split("-")[1])))//2
        except:
            print(row['Survey'])
    else:
        row['Year'] = int(row['Survey'].split()[-1])
    row['Area'] = row['Survey'].split("-")[0]
    
    return row

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data tranfer (pipe) to transfer data between the main process and workers.


In [87]:
for item in tqdm(glob("data/*.csv")):
    print(item)

    data = pd.read_csv(item, encoding='latin-1')
    filecolumns = tuple(data.columns)
    if filecolumns == column_types[4]:
        for key, dataframe in data.groupby(['Indicator', 'Source']):
            var_id = variables[variables['name'] == '%s - %s (FAO (2019))' % (key[0], key[1])]['id'].values[0]
            dataframe.dropna(subset=['Value'], inplace=True)
            dataframe = dataframe.apply(create_var_val_1_2, axis=1) 
            dataframe.rename(columns={"Value": "value", "Area": "country", "Year": "year"}, inplace=True)
            
            dataframe[['value', 'country', 'year']].to_csv('./datapoints/datapoints_%s.csv' % str(var_id), index=False)  
      
    elif filecolumns == column_types[8]:
        for key, dataframe in data.groupby(['Breakdown Variable']):
            var_id = variables[variables['name'] == key + " (FAO (2019))"]['id'].values[0]
            dataframe.dropna(subset=['Value'], inplace=True)
            dataframe = dataframe.apply(create_var_val_8, axis=1) 
            dataframe.rename(columns={"Value": "value", "Area": "country", "Year": "year"}, inplace=True)
            dataframe[['value', 'country', 'year']].to_csv('./datapoints/datapoints_%s.csv' % str(var_id), index=False)
    elif filecolumns == column_types[9]:
        for key, dataframe in data.groupby(['Indicator']):
            var_id = variables[variables['name'] == key + " (FAO (2019))"]['id'].values[0]
            dataframe.dropna(subset=['Value'], inplace=True)
            dataframe = dataframe.apply(create_var_val_1_2, axis=1) 
            dataframe.rename(columns={"Value": "value", "Area": "country", "Year": "year"}, inplace=True)
            dataframe[['value', 'country', 'year']].to_csv('./datapoints/datapoints_%s.csv' % str(var_id), index=False)
    elif filecolumns == column_types[10] or filecolumns == column_types[11] or filecolumns == column_types[5]:
        for key, dataframe in data.groupby(['Item']):
            var_id = variables[variables['name'] == key + " (FAO (2019))"]['id'].values[0]
            dataframe.dropna(subset=['Value'], inplace=True)
            dataframe = dataframe.apply(create_var_val_1_2, axis=1) 
            dataframe.rename(columns={"Value": "value", "Area": "country", "Year": "year", "Recipient Country": "country"}, inplace=True)
            dataframe[['value', 'country', 'year']].to_csv('./datapoints/datapoints_%s.csv' % str(var_id), index=False)
    elif filecolumns == column_types[12]:
        for key, dataframe in data.groupby(['Element']):
            var_id = variables[variables['name'] == key + " (FAO (2019))"]['id'].values[0]
            dataframe.dropna(subset=['Value'], inplace=True)
            dataframe = dataframe.apply(create_var_val_1_2, axis=1) 
            dataframe.rename(columns={"Value": "value", "Area": "country", "Year": "year"}, inplace=True)
            dataframe[['value', 'country', 'year']].to_csv('./datapoints/datapoints_%s.csv' % str(var_id), index=False)  
    
    else:
        
        if item == 'data/Emissions_Agriculture_Energy_E_All_Data_(Norm).csv' or \
                item == 'data/Production_LivestockPrimary_E_All_Data_(Normalized).csv' or item == 'data/Trade_LiveAnimals_E_All_Data_(Normalized).csv':
            for key, dataframe in data.groupby(['Item', 'Element', 'Unit']):
                
                var_id = variables[variables['name'] == '%s - %s - %s (FAO (2019))' % (key[0], key[1], key[2])]['id'].values[0] 
                dataframe.dropna(subset=['Value'], inplace=True)
                dataframe = dataframe.apply(create_var_val_1_2, axis=1) 
                dataframe.rename(columns={"Value": "value", "Area": "country","Country": "country", "Year": "year"}, inplace=True)
                dataframe[['value', 'country', 'year']].to_csv('./datapoints/datapoints_%s.csv' % str(var_id), index=False)
        else:
        
            for key, dataframe in data.groupby(['Item', 'Element']):
                if filecolumns == column_types[0]:
                    var_id = variables[variables['name'] == key[0] + " (FAO (2019))"]['id'].values[0] 
                    dataframe.dropna(subset=['Value'], inplace=True)
                    dataframe = dataframe.apply(create_var_val_1_2, axis=1) 
                    dataframe.rename(columns={"Value": "value", "Area": "country", "Year": "year"}, inplace=True)
                    dataframe[['value', 'country', 'year']].to_csv('./datapoints/datapoints_%s.csv' % str(var_id), index=False)  

                if filecolumns == column_types[1] or filecolumns == column_types[2] or filecolumns == column_types[3]:
                    variablename = '%s - %s (FAO (2019))' % (row['Item'], row['Element'])
                    var_id = variables[variables['name'] == '%s - %s (FAO (2019))' % (key[0], key[1])]['id'].values[0] 
                    dataframe.dropna(subset=['Value'], inplace=True)
                    dataframe = dataframe.apply(create_var_val_1_2, axis=1) 
                    dataframe.rename(columns={"Value": "value", "Area": "country","Country": "country", "Year": "year"}, inplace=True)
                    dataframe[['value', 'country', 'year']].to_csv('./datapoints/datapoints_%s.csv' % str(var_id), index=False)  













  0%|          | 0/79 [00:00<?, ?it/s]

data/Inputs_Pesticides_Use_E_All_Data_(Normalized).csv


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:68: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy











  1%|▏         | 1/79 [00:07<10:05,  7.76s/it]










  3%|▎         | 2/79 [00:07<07:01,  5.48s/it]

data/Environment_AirClimateChange_E_All_Data.csv
data/Emissions_Land_Use_Cropland_E_All_Data_(Normalized).csv













  4%|▍         | 3/79 [00:10<05:52,  4.64s/it]

...












 97%|█████████▋| 77/79 [3:23:34<03:47, 113.75s/it]

data/Trade_DetailedTradeMatrix_E_All_Data_(Normalized).csv













 99%|█████████▊| 78/79 [3:24:27<01:35, 95.63s/it] 

data/FoodBalanceSheets_E_All_Data_(Normalized).csv













100%|██████████| 79/79 [3:49:35<00:00, 174.38s/it]


## Get country names

In [88]:
countries = set()

for x in tqdm(glob('datapoints/*.csv')):
    
   
    data = pd.read_csv(x)
    for j in data['country'].values:
        countries.add(j)
res = pd.DataFrame()
res['name'] = list(countries)
res.to_csv("distinct_countries_standardized.csv", index=False)












  0%|          | 0/9686 [00:00<?, ?it/s]










  0%|          | 1/9686 [00:00<40:35,  3.98it/s]










  0%|          | 16/9686 [00:00<28:42,  5.61it/s]










  0%|          | 40/9686 [00:00<20:15,  7.94it/s]










  1%|          | 61/9686 [00:00<14:23, 11.15it/s]










  1%|          | 81/9686 [00:00<10:17, 15.56it/s]










  1%|          | 100/9686 [00:00<07:26, 21.45it/s]










  1%|          | 120/9686 [00:00<05:27, 29.22it/s]










  1%|▏         | 142/9686 [00:00<04:02, 39.43it/s]










  2%|▏         | 161/9686 [00:01<03:05, 51.46it/s]










  2%|▏         | 180/9686 [00:01<02:24, 65.73it/s]










  2%|▏         | 201/9686 [00:01<01:55, 82.43it/s]










  2%|▏         | 225/9686 [00:01<01:32, 102.21it/s]










  3%|▎         | 246/9686 [00:01<01:19, 118.91it/s]










  3%|▎         | 267/9686 [00:01<01:09, 135.03it/s]










  3%|▎         | 287/9686 [00:01<01:05, 144.24it/s]










  3%|▎         | 307/9686

 38%|███▊      | 3640/9686 [00:18<00:29, 205.35it/s]










 38%|███▊      | 3661/9686 [00:18<00:30, 196.19it/s]










 38%|███▊      | 3681/9686 [00:18<00:30, 195.18it/s]










 38%|███▊      | 3701/9686 [00:18<00:30, 194.48it/s]










 38%|███▊      | 3724/9686 [00:18<00:29, 203.03it/s]










 39%|███▊      | 3745/9686 [00:19<00:29, 199.03it/s]










 39%|███▉      | 3766/9686 [00:19<00:30, 197.25it/s]










 39%|███▉      | 3788/9686 [00:19<00:29, 201.27it/s]










 39%|███▉      | 3809/9686 [00:19<00:29, 198.49it/s]










 40%|███▉      | 3829/9686 [00:19<00:30, 192.19it/s]










 40%|███▉      | 3849/9686 [00:19<00:30, 193.89it/s]










 40%|███▉      | 3869/9686 [00:19<00:31, 187.54it/s]










 40%|████      | 3888/9686 [00:19<00:31, 184.45it/s]










 40%|████      | 3912/9686 [00:19<00:29, 197.41it/s]










 41%|████      | 3938/9686 [00:19<00:27, 210.87it/s]










 41%|████      | 3963/9686 [00:20<00:25, 220.20it/s]




 77%|███████▋  | 7432/9686 [00:36<00:12, 184.25it/s]










 77%|███████▋  | 7452/9686 [00:37<00:11, 188.45it/s]










 77%|███████▋  | 7472/9686 [00:37<00:11, 191.32it/s]










 77%|███████▋  | 7492/9686 [00:37<00:11, 190.59it/s]










 78%|███████▊  | 7514/9686 [00:37<00:10, 198.46it/s]










 78%|███████▊  | 7534/9686 [00:37<00:10, 198.12it/s]










 78%|███████▊  | 7558/9686 [00:37<00:10, 205.95it/s]










 78%|███████▊  | 7579/9686 [00:37<00:10, 206.45it/s]










 78%|███████▊  | 7600/9686 [00:37<00:10, 205.36it/s]










 79%|███████▊  | 7621/9686 [00:37<00:10, 203.06it/s]










 79%|███████▉  | 7642/9686 [00:37<00:10, 198.37it/s]










 79%|███████▉  | 7662/9686 [00:38<00:10, 195.44it/s]










 79%|███████▉  | 7682/9686 [00:38<00:10, 193.37it/s]










 80%|███████▉  | 7702/9686 [00:38<00:10, 191.30it/s]










 80%|███████▉  | 7723/9686 [00:38<00:10, 195.25it/s]










 80%|███████▉  | 7743/9686 [00:38<00:10, 183.92it/s]




## Insert db

In [ ]:
with connection as c:
    db = DBUtils(c)
    
    entities = pd.read_csv("distinct_countries_standardized.csv")
    datasets = pd.read_csv("datasets.csv")
    sources = pd.read_csv("sources.csv")
    variables = pd.read_csv('variables.csv')
    
    new_entities = entities[entities['db_entity_id'].isnull()]
    for _, entity in new_entities.iterrows():
        entity_id = entity.name
        entity_name = entity['name']
        db_entity_id = db.get_or_create_entity(entity_name)
        entities.loc[entity_id, 'db_entity_id'] = db_entity_id
    
    # upsert datasets
    dataset_name_ids = {}
    for i, row in tqdm(datasets.iterrows()):
        dataset_id = db.upsert_dataset(name=row['name'], namespace="faostat", user_id=15)
        dataset_name_ids[row['name']] = dataset_id
        
        
    # upsert sources
    
    dataset_to_source_ids = {}
    for i, row in tqdm(sources.iterrows()):

        dataset_name = datasets[datasets['id'] == row['dataset_id']]['name'].values[0]
        source_id = db.upsert_source(name=row['name'], description=json.dumps(row['description']), dataset_id=dataset_name_ids[dataset_name])

        dataset_to_source_ids[dataset_name] = source_id

        
    # upsert variables
    names_to_ids = {}
    for i, row in tqdm(variables.iterrows()):
        
        dataset_name = datasets[datasets['id'] == row['dataset_id']]['name'].values[0]
        dataset_id = dataset_name_ids[dataset_name]
        source_id = dataset_to_source_ids[dataset_name]
        
        unit = row['unit'] if pd.notnull(row['unit']) else ""
        
        variable_id = db.upsert_variable(
                                        name=row['name'], 
                                        code=None, 
                                        unit=unit, 
                                        short_unit=None, 
                                        source_id=source_id, 
                                        dataset_id=dataset_id, 
                                        description=varname_description[row['name']], 
                                        timespan='', 
                                        coverage='', 
                                        display={}
                                        )
        names_to_ids[row['name']] = variable_id
        
    #Inserting datapoints


    datapoints_files = glob("datapoints/*.csv")
    for x in tqdm(datapoints_files): 

        v_id = int(x.split("_")[1].split(".")[0])
       
        # to get variable name
        variable_name = variables[variables['id']==v_id]['name'].values[0]
       
        # to get variable id from db
        variable_id = names_to_ids[variable_name]
        data = pd.read_csv(x)
        

        for i, row in data.iterrows():
            entity_id = entities[entities['name'] == row['country']]['db_entity_id'].values[0]

            year = row['year']
            val = row['value']
            
            try:

                db.upsert_one("""
                    INSERT INTO data_values
                        (value, year, entityId, variableId)
                    VALUES
                        (%s, %s, %s, %s)
                    ON DUPLICATE KEY UPDATE
                        value = VALUES(value),
                        year = VALUES(year),
                        entityId = VALUES(entityId),
                        variableId = VALUES(variableId)
                """, [val, int(year), str(int(entity_id)), str(variable_id)])
            except:
                print(x)
                print(row['year'], row['value'], entity_id, variable_id)
    

79it [00:00, 1540.89it/s]
79it [00:00, 675.42it/s]
9686it [00:11, 848.96it/s]
  0%|          | 12/9686 [01:20<18:02:38,  6.71s/it]